In [ ]:
# Library Import
# datasets 와 동일한 계층 폴더에서 실행하면 됨
import os
import json
import shutil
from tqdm import tqdm

In [ ]:
def convert_coco_to_yolo_format(root_dir: str, json_file: str, save_dir: str):
# Check directory
    try:
        assert os.path.exists(os.path.join(root_dir, save_dir, "images")) == True
        assert os.path.exists(os.path.join(root_dir, save_dir, "labels")) == True
    except:
        os.makedirs(os.path.join(root_dir, save_dir, "images"), exist_ok=True)
        os.makedirs(os.path.join(root_dir, save_dir, "labels"), exist_ok=True)
    finally:
        print("Finish make directory")

# Load json
    with open(os.path.join(root_dir, json_file), 'r') as f:
        coco_json = json.load(f)
    anoots = coco_json["annotations"]

    print("Start converting...")
    for image in tqdm(sorted(coco_json["images"], key=lambda x: x["id"])):
        w, h, file_name, image_id = image["width"], image["height"], image["file_name"], image["id"]
        file_name = file_name.split("/")[1]
    # filtering annotations
        obj_candits = list(filter(lambda x: x["image_id"] == image_id, anoots))
    # Save txt format to train yolo
        with open(os.path.join(root_dir, save_dir, "labels", f"{file_name[:-4]}.txt"), "w") as f:
            for obj_candit in obj_candits:
             # x1 y1 w h -> cx cy w h               
                cat_id = obj_candit["category_id"]

                x1, y1, width, height = obj_candit["bbox"]
                scaled_cx, scaled_cy = (x1+width/2) / w, (y1+height/2) / h
                scaled_width, scaled_height = width / w, height / h
                f.write("%s %.3f %.3f %.3f %.3f\n" %(cat_id, scaled_cx, scaled_cy, scaled_width, scaled_height))
            f.close()    # Copy image to new directory
        shutil.copy(os.path.join(root_dir, save_dir.split('_')[-1], file_name), os.path.join(root_dir, save_dir, "images", file_name))
    print("Finish converting...")

images, label folder생성

In [44]:
convert_coco_to_yolo_format("../../../dataset", "train.json", "yolo_train")

Finish make directory
Start converting...


 47%|████▋     | 2281/4883 [00:07<00:07, 325.62it/s]


KeyboardInterrupt: 

In [ ]:
convert_coco_to_yolo_format("../../../dataset", "test.json", "yolo_test")

Finish make directory
Start converting...


  0%|          | 0/4871 [00:00<?, ?it/s]

100%|██████████| 4871/4871 [00:01<00:00, 3220.87it/s]

Finish converting...


images, label .txt로 valid 분할

In [51]:
from glob import glob
train_img_list = glob('/data/ephemeral/home/level2-objectdetection-cv-10/dataset/yolo_train/images/*.jpg')
print(len(train_img_list))

4883


In [53]:
from sklearn.model_selection import train_test_split
train_list, valid_list = train_test_split(train_img_list, test_size=0.001, random_state = 2000)
print(len(train_list), len(valid_list))

4878 5


In [54]:
with open('/data/ephemeral/home/level2-objectdetection-cv-10/dataset/yolo_train/train.txt', 'w') as f:
    f.write('\n'.join(train_img_list) + '\n')

In [55]:
# train data에 80%, valid에 20% 배정

with open('/data/ephemeral/home/level2-objectdetection-cv-10/dataset/yolo_train/train.txt', 'w') as f:
    f.write('\n'.join(train_list) + '\n')

with open('/data/ephemeral/home/level2-objectdetection-cv-10/dataset/yolo_train/valid.txt', 'w') as f:
    f.write('\n'.join(valid_list) + '\n')

In [57]:
import yaml

with open('./data/custom_data.yaml','r') as f:
  data = yaml.safe_load(f)

print(data)

data['train'] = '/data/ephemeral/home/level2-objectdetection-cv-10/dataset/yolo_train/train.txt'
data['val'] = '/data/ephemeral/home/level2-objectdetection-cv-10/dataset/yolo_train/valid.txt'
data['test'] = '/data/ephemeral/home/level2-objectdetection-cv-10/dataset/yolo_test/images'

with open('./data/data.yaml', 'w') as f:
  yaml.dump(data, f)

print(data)

{'download': 'bash ./scripts/get_coco.sh', 'names': ['General trash', 'Paper', 'Paper pack', 'Metal', 'Glass', 'Plastic', 'Styrofoam', 'Plastic bag', 'Battery', 'Clothing'], 'nc': 10, 'test': '/data/ephemeral/home/level2-objectdetection-cv-10/dataset/yolo_test/images', 'val': '/data/ephemeral/home/level2-objectdetection-cv-10/dataset/yolo_train/valid.txt', 'train': '/data/ephemeral/home/level2-objectdetection-cv-10/dataset/yolo_train/train.txt'}
{'download': 'bash ./scripts/get_coco.sh', 'names': ['General trash', 'Paper', 'Paper pack', 'Metal', 'Glass', 'Plastic', 'Styrofoam', 'Plastic bag', 'Battery', 'Clothing'], 'nc': 10, 'test': '/data/ephemeral/home/level2-objectdetection-cv-10/dataset/yolo_test/images', 'val': '/data/ephemeral/home/level2-objectdetection-cv-10/dataset/yolo_train/valid.txt', 'train': '/data/ephemeral/home/level2-objectdetection-cv-10/dataset/yolo_train/train.txt'}


모델과 size 비교 !

![](https://miro.medium.com/v2/resize:fit:1100/format:webp/1*kgFvdXMla-1lnTwO8fvZfQ.png)

yolov7x 훈련 코드

In [58]:
!python train.py --device 0 --batch-size 16 --epochs 10 --img 640 640 --data data/custom_data.yaml --hyp data/hyp.scratch.custom.yaml --cfg cfg/training/yolov7x-custom.yaml --weights yolov7x.pt --name yolov7x-custom

YOLOR 🚀 v0.1-128-ga207844 torch 1.12.1+cu116 CUDA:0 (Tesla V100-SXM2-32GB, 32510.5MB)

Namespace(weights='yolov7x.pt', cfg='cfg/training/yolov7x-custom.yaml', data='data/custom_data.yaml', hyp='data/hyp.scratch.custom.yaml', epochs=10, batch_size=16, img_size=[640, 640], rect=False, resume=False, nosave=False, notest=False, noautoanchor=False, evolve=False, bucket='', cache_images=False, image_weights=False, device='0', multi_scale=False, single_cls=False, adam=False, sync_bn=False, local_rank=-1, workers=8, project='runs/train', entity=None, name='yolov7x-custom', exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=-1, artifact_alias='latest', freeze=[0], v5_metric=False, world_size=1, global_rank=-1, save_dir='runs/train/yolov7x-custom13', total_batch_size=16)
tensorboard: Start with 'tensorboard --logdir runs/train', view at http://localhost:6006/
hyperparameters: lr0=0.001, lrf=0.1, momentum=0.937, weight_decay=0.000

yolov7-e6e-custom 훈련코드

In [17]:
!python train_aux.py --device 0 --batch-size 2 --epochs 2 --img 1280 1280 --data data/custom_data.yaml --hyp data/hyp.scratch.custom.yaml --cfg cfg/training/yolov7-e6e-custom.yaml --weights yolov7-e6e.pt --name yolov7-e6e-custom

YOLOR 🚀 v0.1-128-ga207844 torch 1.12.1+cu116 CUDA:0 (Tesla V100-SXM2-32GB, 32510.5MB)

Namespace(weights='yolov7-e6e.pt', cfg='cfg/training/yolov7-e6e-custom.yaml', data='data/custom_data.yaml', hyp='data/hyp.scratch.custom.yaml', epochs=2, batch_size=2, img_size=[1280, 1280], rect=False, resume=False, nosave=False, notest=False, noautoanchor=False, evolve=False, bucket='', cache_images=False, image_weights=False, device='0', multi_scale=False, single_cls=False, adam=False, sync_bn=False, local_rank=-1, workers=8, project='runs/train', entity=None, name='yolov7-e6e-custom', exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=-1, artifact_alias='latest', v5_metric=False, world_size=1, global_rank=-1, save_dir='runs/train/yolov7-e6e-custom12', total_batch_size=2)
tensorboard: Start with 'tensorboard --logdir runs/train', view at http://localhost:6006/
hyperparameters: lr0=0.001, lrf=0.1, momentum=0.937, weight_decay=0.0005

inference 코드

In [5]:
#!python test.py --device 0 --batch-size 16 --img-size 640 --data data/custom_data.yaml --conf-thres 0.001 --iou-thres 0.45 --weights runs/train/yolov7x-custom6/weights/best.pt --name yolov7_7x_test_data

Namespace(weights=['runs/train/yolov7x-custom6/weights/best.pt'], data='data/custom_data.yaml', batch_size=16, img_size=640, conf_thres=0.001, iou_thres=0.45, task='val', device='0', single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project='runs/test', name='yolov7_7x_test_data', exist_ok=False, no_trace=False, v5_metric=False)
YOLOR 🚀 v0.1-128-ga207844 torch 1.12.1+cu116 CUDA:0 (Tesla V100-SXM2-32GB, 32510.5MB)

^C


In [16]:
!python detect.py --device 0 --img-size 640 --conf-thres 0.01 --iou-thres 0.45 --nosave --weights runs/train/yolov7x-custom6/weights/best.pt --source /data/ephemeral/home/level2-objectdetection-cv-10/dataset/yolo_test/images --name yolov7_x1_test_data

Namespace(weights=['runs/train/yolov7x-custom6/weights/best.pt'], source='/data/ephemeral/home/level2-objectdetection-cv-10/dataset/yolo_test/images', img_size=640, conf_thres=0.01, iou_thres=0.45, device='0', view_img=False, save_txt=False, save_conf=False, nosave=True, classes=None, agnostic_nms=False, augment=False, update=False, project='runs/detect', name='yolov7_x1_test_data', exist_ok=False, no_trace=False)
YOLOR 🚀 v0.1-128-ga207844 torch 1.12.1+cu116 CUDA:0 (Tesla V100-SXM2-32GB, 32510.5MB)

Fusing layers... 
IDetect.fuse
/opt/conda/lib/python3.10/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 362 layers, 70843086 parameters, 0 gradients, 188.2 GFLOPS
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is tr